In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import math
import folium
from folium import GeoJson

In [2]:
# try and reduce the amount of times you run this itll take forever.
gdf = gpd.read_file('.\output.geojson')

In [3]:
def remove_times():
    # use for error in folio TypeError: Object of type Timestamp is not JSON serializable
    drop_ls = []
    for c in gdf.columns:
        # gns:n:xx:modify_date
        # gns:v:xx:modify_date
        if ("time" in c or "date" in c or "hour" in c):
            print(c)
            drop_ls.append(c)

    return gdf.drop(drop_ls, axis=1)

gdf = remove_times() # only use if you have an error with folium Geo.Json

check_date
survey:date


In [4]:
startDt = pd.read_csv(".\May2023.csv")
endDt = pd.read_csv(".\May2024.csv")
startDt['location'] = startDt.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
endDt['location'] = endDt.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

In [5]:
# get the center of all unique munis

def get_center_loc(group, id=None, res_dic=None):
    # group is a data frame
    # source http://www.geomidpoint.com/example.html
    # gets the center lat lng
    # assuming the earth is perfectly round
    rads_lat = np.radians(group.latitude)
    rads_lon = np.radians(group.longitude)
    
    # Convert lat/lon (radians) to Cartesian coordinates for each location.
    X = np.cos(rads_lat) * np.cos(rads_lon)
    Y = np.cos(rads_lat) * np.sin(rads_lon)
    Z = np.sin(rads_lat)

    #find average x, y, z coords
    x = X.mean()
    y = Y.mean()
    z = Z.mean()

    # Convert average x, y, z coordinate to latitude and longitude.
    lng = math.atan2(y, x)
    hyp = np.sqrt(x*x + y*y)
    lat = math.atan2(z, hyp)
    if res_dic:
        res_dic[id] = (np.degrees(lat), np.degrees(lng))
    else:
        return (np.degrees(lat), np.degrees(lng))
    
result = {}
startDt.groupby(['neighbourhood_group']).apply(lambda group: get_center_loc(group=group, id=group.name, res_dic=result))


C:\Users\Alex\AppData\Local\Temp\ipykernel_11348\1912456642.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  startDt.groupby(['neighbourhood_group']).apply(lambda group: get_center_loc(group=group, id=group.name, res_dic=result))


neighbourhood_group
Central Saanich            (48.5790387262022, -123.43224583046444)
Colwood                   (48.42056053099835, -123.49314834500264)
Esquimalt                  (48.4327476950025, -123.40825038883719)
Highlands                  (48.49759222145811, -123.5064005885147)
Juan de Fuca             (48.450202069477285, -123.97807266932395)
Langford                  (48.44891934602266, -123.51647680896993)
Metchosin                 (48.38116826582552, -123.54185860487911)
North Saanich             (48.65907421230939, -123.43781351793487)
Oak Bay                   (48.43170969865602, -123.31458997323222)
Saanich                  (48.477644462340436, -123.36273107107148)
Salt Spring Island         (48.84870481330521, -123.5068431206925)
Sidney                    (48.65489185867977, -123.40392673779033)
Sooke                    (48.379830875237296, -123.71513428104477)
Southern Gulf Islands      (48.8268471960932, -123.30783288457276)
Victoria                 (48.4255836737468

In [6]:
# break up all the startDt data into single owners and multi owners
start_count = startDt.host_id.value_counts()
multi_start = startDt[startDt.host_id.isin(start_count.index[start_count.gt(1)])]
single_start = startDt[startDt.host_id.isin(start_count.index[start_count.lt(2)])]

# break up all endDt data into multi owners and single owners
end_count = endDt.host_id.value_counts()
multi_end = endDt[endDt.host_id.isin(end_count.index[end_count.gt(1)])]
single_end = endDt[endDt.host_id.isin(end_count.index[end_count.lt(2)])]

In [7]:
# get mean price grouped by host_id
# values tracking multi_start, multi_end, single_start, single_end, singleDt, endDt
multi_start['mean_price'] = multi_start.groupby(['host_id']).price.transform('mean')
multi_end['mean_price'] = multi_end.groupby(['host_id']).price.transform('mean')

C:\Users\Alex\AppData\Local\Temp\ipykernel_11348\304606603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_start['mean_price'] = multi_start.groupby(['host_id']).price.transform('mean')
C:\Users\Alex\AppData\Local\Temp\ipykernel_11348\304606603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_end['mean_price'] = multi_end.groupby(['host_id']).price.transform('mean')


In [94]:
from folium.plugins import HeatMap
map = folium.Map(location=get_center_loc(startDt), prefer_canvas=True, tiles="cartodb positron", zoom_start=9)

multi_end.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=2,
                                popup=x.price,
                                color="red",
                            ).add_to(map), axis=1)

single_end.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=2,
                                popup=x.price,
                                color="blue",
                            ).add_to(map), axis=1)

    

# get real center for group listings
# this function could be cleaned up
def get_group_center(group, center):
    i = 0
    for index, row in group.iterrows():
        if i == 0:
            closest = (row.latitude, row.longitude)
            i += 1
        if abs(row.latitude - center[row.host_name][0] + row.longitude - center[row.host_name][1]) < abs(closest[0]-center[row.host_name][0] + closest[1]-center[row.host_name][1]):
            closest = (row.latitude, row.longitude)
    return closest

# find closest to center listing
def find_center_listing(df):
    center_listing = {}
    center = df.groupby(['host_name']).apply(get_center_loc)
    for host in df.host_name.unique():
        center_listing[host] = get_group_center(df[df.host_name == host], center)
    
    return center_listing

multi_start_centers = find_center_listing(multi_start)
multi_end_centers = find_center_listing(multi_end)


set(multi_start_centers) ^  set(multi_end_centers))

# test
# HeatMap(data=startDt.locations.tolist()).add_to(map)

# folium.LayerControl().add_to(map)
                                
# map

C:\Users\Alex\AppData\Local\Temp\ipykernel_11348\1683895927.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  center = df.groupby(['host_name']).apply(get_center_loc)
C:\Users\Alex\AppData\Local\Temp\ipykernel_11348\1683895927.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  center = df.groupby(['host_name']).apply(get_center_loc)


447

In [256]:
c= pd.DataFrame({
  "host_name": ['a','a','b'],
  "latitude": [1, 2, 3],
  "longitude": [10, 20, 30]
})

center = c.groupby(['host_name']).apply(get_center_loc)
c.groupby(['host_name']).apply(get_group_center, 

C:\Users\Alex\AppData\Local\Temp\ipykernel_3164\1020284990.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  center = c.groupby(['host_name']).apply(get_center_loc)
